In [2]:
from ultralytics import YOLO
import cv2

model = YOLO('yolov8n.pt')
cap = cv2.VideoCapture("IMG_3583.mp4")

line_y = 760
entry_line = [(840, line_y), (1250, line_y)]

track_memory = {}
count_in = 0  
count_out = 0  
counted_ids = set()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(frame, persist=True, classes=[0], conf=0.4)
    frame_copy = frame.copy()

    if results[0].boxes.id is not None:
        ids = results[0].boxes.id.cpu().numpy()
        boxes = results[0].boxes.xyxy.cpu().numpy()

        for track_id, box in zip(ids, boxes):
            x1, y1, x2, y2 = box.astype(int)
            cx = int((x1 + x2) / 2)
            cy = int(y2) 


            cv2.rectangle(frame_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.circle(frame_copy, (cx, cy), 4, (0, 255, 255), -1)

            if track_id not in track_memory:
                track_memory[track_id] = [cy]
            else:
                track_memory[track_id].append(cy)
                if len(track_memory[track_id]) >= 2 and track_id not in counted_ids:
                    dy = cy - track_memory[track_id][-2]

                    if track_memory[track_id][-2] > line_y >= cy and dy < 0:
                        count_in += 1
                        counted_ids.add(track_id)

                    elif track_memory[track_id][-2] < line_y <= cy and dy > 0:
                        count_out += 1
                        counted_ids.add(track_id)


                track_memory[track_id] = track_memory[track_id][-2:]


    cv2.line(frame_copy, entry_line[0], entry_line[1], (255, 0, 0), 2)
    cv2.putText(frame_copy, f'IN: {count_in}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame_copy, f'OUT: {count_out}', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Counting People", frame_copy)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 6 persons, 175.9ms
Speed: 8.0ms preprocess, 175.9ms inference, 18.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 102.8ms
Speed: 3.8ms preprocess, 102.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 68.1ms
Speed: 4.3ms preprocess, 68.1ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 75.1ms
Speed: 2.7ms preprocess, 75.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 68.5ms
Speed: 3.6ms preprocess, 68.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 67.3ms
Speed: 4.2ms preprocess, 67.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 62.9ms
Speed: 4.4ms preprocess, 62.9ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 66.9ms
Speed: 2.8ms preprocess, 66.9ms inference, 1.1ms postprocess per image at sh

In [20]:
from ultralytics import YOLO

# โหลดโมเดล YOLOv8 Nano (เล็ก+เร็ว) สำหรับฝึกใหม่
model = YOLO('yolov8n.pt')

# ฝึกโมเดลด้วย dataset ที่ระบุใน data.yaml
results = model.train(
    data='data.yaml',   # ไฟล์ config dataset (ปรับ path ตามไฟล์ของคุณ)
    epochs=50,          # จำนวนรอบการฝึก
    imgsz=640,          # ขนาดภาพ input
    batch=16,           # batch size ปรับตามสเปคเครื่อง
    lr0=0.01,           # learning rate เริ่มต้น
    workers=4,          # จำนวน worker สำหรับโหลดข้อมูล
    name='mrt_door_detect'  # ชื่อโปรเจกต์
)

# หลังจบ training โมเดลจะถูกบันทึกใน
# runs/train/mrt_door_detect/weights/best.pt

New https://pypi.org/project/ultralytics/8.3.135 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.118  Python-3.11.9 torch-2.7.0+cpu CPU (AMD Ryzen 5 4600H with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=None, name=mrt_door_detect13, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, sav

train: Scanning E:\mrt project\datasets\train\labels.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.10.0 ms, read: 210.727.9 MB/s, size: 52.6 KB)



C:\Users\thuns\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning E:\mrt project\datasets\valid\labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3/3 [00:00<?, ?it/s]

Plotting labels to runs\detect\mrt_door_detect13\labels.jpg... 


...

      50/50         0G     0.5857       1.38      1.133          9        640: 100%|██████████| 2/2 [00:07<00:00,  3.68s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.37it/s]

                   all          3          3      0.984          1      0.995      0.929



50 epochs completed in 0.115 hours.
Optimizer stripped from runs\detect\mrt_door_detect13\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\mrt_door_detect13\weights\best.pt, 6.2MB

Validating runs\detect\mrt_door_detect13\weights\best.pt...
Ultralytics 8.3.118  Python-3.11.9 torch-2.7.0+cpu CPU (AMD Ryzen 5 4600H with Radeon Graphics)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.71it/s]


                   all          3          3      0.984          1      0.995      0.929
           door_closed          3          3      0.984          1      0.995      0.929
Speed: 2.5ms preprocess, 78.8ms inference, 0.0ms loss, 30.9ms postprocess per image
Results saved to runs\detect\mrt_door_detect13


In [ ]:
# from ultralytics import YOLO
# import cv2

# # โหลดโมเดล
# person_model = YOLO('yolov8n.pt')       # ตรวจจับคน
# door_model = YOLO('best.pt')            # ตรวจจับประตูเปิด-ปิด (ที่คุณเทรนไว้)

# cap = cv2.VideoCapture("IMG_3583.mp4")

# line_y = 760
# entry_line = [(840, line_y), (1250, line_y)]

# track_memory = {}
# count_in = 0  
# count_out = 0  
# counted_ids = set()

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     frame_copy = frame.copy()

#     # ตรวจประตู
#     door_results = door_model.predict(frame, conf=0.4)
#     door_is_open = False
#     if len(door_results) > 0 and door_results[0].boxes.cls is not None:
#         classes = door_results[0].boxes.cls.cpu().numpy()
#         for cls in classes:
#             if int(cls) == 0:  # สมมุติ class 0 = door_open
#                 door_is_open = True
#                 break

#     # ตรวจคน
#     results = person_model.track(frame, persist=True, classes=[0], conf=0.4)

#     if results[0].boxes.id is not None:
#         ids = results[0].boxes.id.cpu().numpy()
#         boxes = results[0].boxes.xyxy.cpu().numpy()

#         for track_id, box in zip(ids, boxes):
#             x1, y1, x2, y2 = box.astype(int)
#             cx = int((x1 + x2) / 2)
#             cy = int(y2)

#             cv2.rectangle(frame_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)
#             cv2.circle(frame_copy, (cx, cy), 4, (0, 255, 255), -1)

#             if track_id not in track_memory:
#                 track_memory[track_id] = [cy]
#             else:
#                 track_memory[track_id].append(cy)
#                 if len(track_memory[track_id]) >= 2 and track_id not in counted_ids and door_is_open:
#                     dy = cy - track_memory[track_id][-2]

#                     if track_memory[track_id][-2] > line_y >= cy and dy < 0:
#                         count_in += 1
#                         counted_ids.add(track_id)

#                     elif track_memory[track_id][-2] < line_y <= cy and dy > 0:
#                         count_out += 1
#                         counted_ids.add(track_id)

#                 track_memory[track_id] = track_memory[track_id][-2:]

#     cv2.line(frame_copy, entry_line[0], entry_line[1], (255, 0, 0), 2)
#     cv2.putText(frame_copy, f'IN: {count_in}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#     cv2.putText(frame_copy, f'OUT: {count_out}', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
#     cv2.putText(frame_copy, f'Door: {"OPEN" if door_is_open else "CLOSED"}', (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

#     cv2.imshow("Counting People", frame_copy)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()



0: 384x640 (no detections), 58.7ms
Speed: 3.1ms preprocess, 58.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 50.7ms
Speed: 1.9ms preprocess, 50.7ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 67.3ms
Speed: 2.5ms preprocess, 67.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 54.6ms
Speed: 2.4ms preprocess, 54.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 47.7ms
Speed: 2.6ms preprocess, 47.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 51.5ms
Speed: 2.0ms preprocess, 51.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 45.4ms
Speed: 2.3ms preprocess, 45.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 48.2ms
Speed: 1.8ms preprocess, 48.2ms inference, 1.1ms postproc